# 網路爬蟲_104人力銀行
找工作真的是很辛苦的一件事情，在人力銀行網站上查詢職缺時，
推薦系統的考量標準並不完全是按照輸入的關鍵字與職缺多適配（這中間還牽涉到公司為了增加曝光度而投遞廣告），
此外也很容易跳出許多不符合我們需求的資訊，而當我們想要濾除這些資訊時，還會發現濾除條件的功能也並不這麼友善...
在這裡我嘗試透過Python的爬蟲，只要使用者輸入查詢的關鍵字，就將查詢出的結果匯出成excel，讓我們不用再一個一個點進去看。
那麼我們就開始吧！

In [1]:
import pandas as pd
import re, time, requests
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
# 加入使用者資訊(如使用什麼瀏覽器、作業系統...等資訊)模擬真實瀏覽網頁的情況
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

# 查詢的關鍵字
my_params = {'ro':'1', # 限定全職的工作，如果不限定則輸入0
             'keyword':'資料科學', # 想要查詢的關鍵字
             'area':'6001001000', # 限定在台北的工作
             'isnew':'30', # 只要最近一個月有更新的過的職缺
             'mode':'l'} # 清單的瀏覽模式

In [7]:
# 透過Selenium打開瀏覽器
url = requests.get('https://www.104.com.tw/jobs/search/?' , my_params, headers = headers).url
driver = webdriver.Chrome()
driver.get(url)

# 網頁的設計方式是滑動到下方時，會自動加載新資料，在這裡透過程式送出Java語法幫我們執行「滑到下方」的動作
for i in range(20): 
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.6)
    
# 自動加載只會加載15次，超過之後必須要點選「手動載入」的按鈕才會繼續載入新資料（可能是防止爬蟲）
k = 1
while k != 0:
    try:
        # 手動載入新資料之後會出現新的more page，舊的就無法再使用，所以要使用最後一個物件
        driver.find_elements_by_class_name("js-more-page",)[-1].click() 
        # 如果真的找不到，也可以直接找中文!
        # driver.find_element_by_xpath("//*[contains(text(),'手動載入')]").click()
        print('Click 手動載入，' + '載入第' + str(15 + k) + '頁')
        k = k+1
        time.sleep(1) # 時間設定太短的話，來不及載入新資料就會跳錯誤
    except:
        k = 0
        print('No more Job')

# 透過BeautifulSoup解析資料
soup = BeautifulSoup(driver.page_source, 'html.parser')
List = soup.findAll('a',{'class':'js-job-link'})
print('共有 ' + str(len(List)) + ' 筆資料')

Click 手動載入，載入第16頁
Click 手動載入，載入第17頁
Click 手動載入，載入第18頁
Click 手動載入，載入第19頁
Click 手動載入，載入第20頁
Click 手動載入，載入第21頁
Click 手動載入，載入第22頁
Click 手動載入，載入第23頁
Click 手動載入，載入第24頁
Click 手動載入，載入第25頁
No more Job
共有 731 筆資料


In [8]:
# 職務類別是複選題，在這裡預先定義一個函數將資料串在一起
def bind(cate):
    k = []
    for i in cate:
        if len(i.text) > 0:
            k.append(i.text)
    return str(k)

In [9]:
JobList = pd.DataFrame()

i = 0
while i < len(List):
    # print('正在處理第' + str(i) + '筆，共 ' + str(len(List)) + ' 筆資料')
    content = List[i]
    # 這裡用Try的原因是，有時候爬太快會遭到系統阻擋導致失敗。因此透過這個方式，當我們遇到錯誤時，會重新再爬一次資料！
    try:
        resp = requests.get('https://' + content.attrs['href'].strip('//'))
        soup2 = BeautifulSoup(resp.text,'html.parser')
        df = pd.DataFrame(
            data = [{
                '公司名稱':soup2.find('a', {'class':'cn'}).text,
                '工作職稱':content.attrs['title'],
                '工作內容':soup2.find('p').text,
                '職務類別':bind(soup2.findAll('dd', {'class':'cate'})[0].findAll('span')),
                '工作待遇':soup2.find('dd', {'class':'salary'}).text.split('\n\n',2)[0].replace(' ',''),
                '工作性質':soup2.select('div > dl > dd')[2].text,
                '上班地點':soup2.select('div > dl > dd')[3].text.split('\n\n',2)[0].split('\n',2)[1].replace(' ',''),
                '管理責任':soup2.select('div > dl > dd')[4].text,
                '出差外派':soup2.select('div > dl > dd')[5].text,
                '上班時段':soup2.select('div > dl > dd')[6].text,
                '休假制度':soup2.select('div > dl > dd')[7].text,
                '可上班日':soup2.select('div > dl > dd')[8].text,
                '需求人數':soup2.select('div > dl > dd')[9].text,
                '接受身份':soup2.select('div.content > dl > dd')[10].text,
                '學歷要求':soup2.select('div.content > dl > dd')[12].text,
                '工作經歷':soup2.select('div.content > dl > dd')[11].text,
                '語文條件':soup2.select('div.content > dl > dd')[14].text,
                '擅長工具':soup2.select('div.content > dl > dd')[15].text,
                '工作技能':soup2.select('div.content > dl > dd')[16].text,
                '其他條件':soup2.select('div.content > dl > dd')[17].text,
                '公司福利':soup2.select('div.content > p')[1].text,
                '科系要求':soup2.select('div.content > dl > dd')[13].text,
                '聯絡方式':soup2.select('div.content')[3].text.replace('\n',''),
                '連結路徑':'https://' + content.attrs['href'].strip('//')}],
            columns = ['公司名稱','工作職稱','工作內容','職務類別','工作待遇','工作性質','上班地點','管理責任','出差外派',
                       '上班時段','休假制度','可上班日','需求人數','接受身份','學歷要求','工作經歷','語文條件','擅長工具',
                       '工作技能','其他條件','公司福利','科系要求','聯絡方式','連結路徑'])
        JobList = JobList.append(df, ignore_index=True)
        i += 1
        print("Success and Crawl Next 目前正在爬第" + str(i) + "個職缺資訊")
        time.sleep(0.5) # 執行完休息0.5秒，避免造成對方主機負擔
    except:
        print("Fail and Try Again!")

Success and Crawl Next 目前正在爬第1個職缺資訊
Success and Crawl Next 目前正在爬第2個職缺資訊
Success and Crawl Next 目前正在爬第3個職缺資訊
Success and Crawl Next 目前正在爬第4個職缺資訊
Success and Crawl Next 目前正在爬第5個職缺資訊
Success and Crawl Next 目前正在爬第6個職缺資訊
Success and Crawl Next 目前正在爬第7個職缺資訊
Success and Crawl Next 目前正在爬第8個職缺資訊
Success and Crawl Next 目前正在爬第9個職缺資訊
Success and Crawl Next 目前正在爬第10個職缺資訊
Success and Crawl Next 目前正在爬第11個職缺資訊
Success and Crawl Next 目前正在爬第12個職缺資訊
Success and Crawl Next 目前正在爬第13個職缺資訊
Success and Crawl Next 目前正在爬第14個職缺資訊
Success and Crawl Next 目前正在爬第15個職缺資訊
Success and Crawl Next 目前正在爬第16個職缺資訊
Success and Crawl Next 目前正在爬第17個職缺資訊
Success and Crawl Next 目前正在爬第18個職缺資訊
Success and Crawl Next 目前正在爬第19個職缺資訊
Success and Crawl Next 目前正在爬第20個職缺資訊
Success and Crawl Next 目前正在爬第21個職缺資訊
Success and Crawl Next 目前正在爬第22個職缺資訊
Success and Crawl Next 目前正在爬第23個職缺資訊
Success and Crawl Next 目前正在爬第24個職缺資訊
Success and Crawl Next 目前正在爬第25個職缺資訊
Success and Crawl Next 目前正在爬第26個職缺資訊
Success and Crawl Next 目前正在爬第27個職缺資訊
Success an

Success and Crawl Next 目前正在爬第214個職缺資訊
Success and Crawl Next 目前正在爬第215個職缺資訊
Success and Crawl Next 目前正在爬第216個職缺資訊
Success and Crawl Next 目前正在爬第217個職缺資訊
Success and Crawl Next 目前正在爬第218個職缺資訊
Success and Crawl Next 目前正在爬第219個職缺資訊
Success and Crawl Next 目前正在爬第220個職缺資訊
Success and Crawl Next 目前正在爬第221個職缺資訊
Success and Crawl Next 目前正在爬第222個職缺資訊
Success and Crawl Next 目前正在爬第223個職缺資訊
Success and Crawl Next 目前正在爬第224個職缺資訊
Success and Crawl Next 目前正在爬第225個職缺資訊
Success and Crawl Next 目前正在爬第226個職缺資訊
Success and Crawl Next 目前正在爬第227個職缺資訊
Success and Crawl Next 目前正在爬第228個職缺資訊
Success and Crawl Next 目前正在爬第229個職缺資訊
Success and Crawl Next 目前正在爬第230個職缺資訊
Success and Crawl Next 目前正在爬第231個職缺資訊
Success and Crawl Next 目前正在爬第232個職缺資訊
Success and Crawl Next 目前正在爬第233個職缺資訊
Success and Crawl Next 目前正在爬第234個職缺資訊
Success and Crawl Next 目前正在爬第235個職缺資訊
Success and Crawl Next 目前正在爬第236個職缺資訊
Success and Crawl Next 目前正在爬第237個職缺資訊
Success and Crawl Next 目前正在爬第238個職缺資訊
Success and Crawl Next 目前正在爬第239個職缺資訊
Success and 

Success and Crawl Next 目前正在爬第429個職缺資訊
Success and Crawl Next 目前正在爬第430個職缺資訊
Success and Crawl Next 目前正在爬第431個職缺資訊
Success and Crawl Next 目前正在爬第432個職缺資訊
Success and Crawl Next 目前正在爬第433個職缺資訊
Success and Crawl Next 目前正在爬第434個職缺資訊
Success and Crawl Next 目前正在爬第435個職缺資訊
Success and Crawl Next 目前正在爬第436個職缺資訊
Success and Crawl Next 目前正在爬第437個職缺資訊
Success and Crawl Next 目前正在爬第438個職缺資訊
Success and Crawl Next 目前正在爬第439個職缺資訊
Success and Crawl Next 目前正在爬第440個職缺資訊
Success and Crawl Next 目前正在爬第441個職缺資訊
Success and Crawl Next 目前正在爬第442個職缺資訊
Success and Crawl Next 目前正在爬第443個職缺資訊
Success and Crawl Next 目前正在爬第444個職缺資訊
Success and Crawl Next 目前正在爬第445個職缺資訊
Success and Crawl Next 目前正在爬第446個職缺資訊
Success and Crawl Next 目前正在爬第447個職缺資訊
Success and Crawl Next 目前正在爬第448個職缺資訊
Success and Crawl Next 目前正在爬第449個職缺資訊
Success and Crawl Next 目前正在爬第450個職缺資訊
Success and Crawl Next 目前正在爬第451個職缺資訊
Success and Crawl Next 目前正在爬第452個職缺資訊
Success and Crawl Next 目前正在爬第453個職缺資訊
Success and Crawl Next 目前正在爬第454個職缺資訊
Success and 

Fail and Try Again!
Fail and Try Again!
Success and Crawl Next 目前正在爬第640個職缺資訊
Success and Crawl Next 目前正在爬第641個職缺資訊
Success and Crawl Next 目前正在爬第642個職缺資訊
Success and Crawl Next 目前正在爬第643個職缺資訊
Fail and Try Again!
Success and Crawl Next 目前正在爬第644個職缺資訊
Success and Crawl Next 目前正在爬第645個職缺資訊
Success and Crawl Next 目前正在爬第646個職缺資訊
Fail and Try Again!
Success and Crawl Next 目前正在爬第647個職缺資訊
Fail and Try Again!
Success and Crawl Next 目前正在爬第648個職缺資訊
Success and Crawl Next 目前正在爬第649個職缺資訊
Success and Crawl Next 目前正在爬第650個職缺資訊
Success and Crawl Next 目前正在爬第651個職缺資訊
Success and Crawl Next 目前正在爬第652個職缺資訊
Success and Crawl Next 目前正在爬第653個職缺資訊
Success and Crawl Next 目前正在爬第654個職缺資訊
Success and Crawl Next 目前正在爬第655個職缺資訊
Success and Crawl Next 目前正在爬第656個職缺資訊
Success and Crawl Next 目前正在爬第657個職缺資訊
Success and Crawl Next 目前正在爬第658個職缺資訊
Success and Crawl Next 目前正在爬第659個職缺資訊
Success and Crawl Next 目前正在爬第660個職缺資訊
Success and Crawl Next 目前正在爬第661個職缺資訊
Success and Crawl Next 目前正在爬第662個職缺資訊
Success and Crawl Next 目前正

In [8]:
JobList

,公司名稱,工作職稱,工作內容,職務類別,工作待遇,工作性質,上班地點,管理責任,出差外派,上班時段,...,學歷要求,工作經歷,語文條件,擅長工具,工作技能,其他條件,公司福利,科系要求,聯絡方式,連結路徑
0,香港商慧科訊業有限公司台灣分公司,大數據後端工程師/軟體研發工程師(台北),大數據後端工程師/軟體研發工程師\r 我們正在尋找對於開發大數據及AI產品充滿熱情，並且樂於...,"['軟體設計工程師', 'Internet程式設計師', '電腦系統分析師']",待遇面議（經常性薪資達4萬元或以上）,全職,台北市信義區基隆路一段396號9樓,不需負擔管理責任,無需出差外派,日班,...,大學、碩士,3年以上,不拘,Linux、Java、Python、AWS,系統架構規劃、軟體程式設計、網路程式設計,需求條件：電機/資訊科學/資訊工程相關學科，學士以上\r1. 三年以上Java 或 Pyth...,1. 週休二日\r2. 人性化管理、安全舒適之工作環境，辦公室設於台北市及台中市精華地段\r...,資訊工程相關、其他數學及電算機科學相關、電機電子工程相關,聯絡人：黃小姐,https://www.104.com.tw/job/?jobno=5jkl3&jobsou...
1,景迅科技有限公司,系統工程師,我們位於內湖科學園區，交通方便，離港墘捷運站步行3分鐘即可到達。\r 工作內容：\r1. 1...,"['網路管理工程師', '系統維護／操作人員', '資訊設備管制人員']",待遇面議（經常性薪資達4萬元或以上）,全職,台北市內湖區洲子街55號11樓(內湖科技園區),不需負擔管理責任,無需出差外派,日班/晚班/假日班，需輪班,...,專科以上,1年以上,不拘,Linux、Perl,不拘,加分條件\r1. 具備 Shell script / Perl / Python 其一程式語...,◆ 獎金制度 \r1.保障年薪14個月（按照到職日依比例計算）\r2.績效獎金（視年度營運及...,不拘,聯絡人：HR,https://www.104.com.tw/job/?jobno=5qm40&jobsou...
2,香港商慧科訊業有限公司台灣分公司,Senior Backend Engineer資深後端工程師(台北),【工作說明】 \r1.\t使用最新技術設計並實現伺服器端應用程式介面API。\r2.\t針...,"['軟體設計工程師', 'Internet程式設計師', '電腦系統分析師']",待遇面議（經常性薪資達4萬元或以上）,全職,台北市信義區基隆路一段396號9樓,不需負擔管理責任,無需出差外派,日班,...,大學、碩士,3年以上,不拘,Linux、Java、Python、MySQL、Oracle,系統架構規劃、軟體程式設計、網路程式設計、資料庫軟體應用,1.\t優秀快速的學習能力，對學習新技術充滿熱情。\r2.\t能用中文有效溝通與書面交流與表...,1. 週休二日\r2. 人性化管理、安全舒適之工作環境，辦公室設於台北市及台中市精華地段\r...,資訊工程相關、其他數學及電算機科學相關、電機電子工程相關,聯絡人：黃小姐,https://www.104.com.tw/job/?jobno=6kg3h&jobsou...
3,易享科技股份有限公司,【新鮮人100K起薪】人才培訓計劃：Android Engineer,MixerBox 目前已於全球突破一億下載量，並獲得矽谷知名加速器 Y Combinator...,['軟體設計工程師'],"月薪100,000元",全職,台北市中正區,不需負擔管理責任,無需出差外派,日班,...,大學以上,不拘,不拘,不拘,不拘,未填寫,◆ 分紅 / 配股\r\t1. 正職員工認股\r◆ 請 / 休假制度\r\t1. 週休二日\...,資訊工程相關、資訊管理相關、其他數學及電算機科學相關,聯絡人：Emily其他：請直接透過104應徵或將簡歷及作品集來信至 mingchentsai...,https://www.104.com.tw/job/?jobno=4hxqf&jobsou...
4,和泰聯網股份有限公司,[和泰汽車集團 和泰聯網]資料科學家,1.\t對機器學習(Machine Learning)、深度學習(Deep Learning...,"['其他資訊專業人員', '電腦系統分析師', '演算法開發工程師']",待遇面議（經常性薪資達4萬元或以上）,全職,台北市中山區松江路433號12樓,不需負擔管理責任,無需出差外派,日班，08:30-17:30,...,碩士以上,3年以上,不拘,Java、Python、MS SQL、Excel,不拘,普通小型車,1.上班地點交通便利，舒適辦公環境\r2.完善福利制度含三節生日禮金及員工團體保險\r3.休...,數理統計相關、資訊工程相關、資訊管理相關,聯絡人：韓先生電洽：,https://www.104.com.tw/job/?jobno=5z4u2&jobsou...
5,慧邦科技股份有限公司,程式設計師,1.程式開發與遊戲維護(PHP + Perl + MySQL)\r2.網路遊戲的前端架構 (...,"['電玩程式設計師', 'Internet程式設計師', '軟體設計工程師']",待遇面議（經常性薪資達4萬元或以上）,全職,台北市中正區羅斯福路2段100號10樓,不需負擔管理責任,無需出差外派,日班，09:30~18:00,...,專科、大學、碩士,不拘,不拘,Perl、PHP、MySQL、JavaScript、jQuery,軟體程式設計,※加分條件：\r- 有使用JavaScript 開發框架經驗 \r- 熟悉 RWD Fram...,熱情有幹勁的員工是我們最大的資產！歡迎加入一起學習成長的環境~\r◆ 交通便利(捷運古亭站出...,電機電子工程相關、資訊工程相關、數學及電算機科學學科類,聯絡人：余小姐,https://www.104.com.tw/job/?jobno=5brs5&jobsou...
6,易享科技股份有限公司,【新鮮人100K起薪】人才培訓計劃：iOS Engineer,由哈佛、台大、政大生打造的 MixerBox 目前已於全球突破一億下載量，並獲得矽谷知名加速...,['軟體設計工程師'],"月薪100,000元",全職,台北市中正區,不需負擔管理責任,無需出差外派,日班,...,大學以上,不拘,不拘,不拘,不拘,未填寫,◆ 分紅 / 配股\r\t1. 正職員工認股\r◆ 請 / 休假制度\r\t1. 週休二日\...,資訊工程相關、資訊管理相關、其他數學及電算機科學相關,聯絡人：Emily其他：請直接透過104應徵或將簡歷及作品集來信至 mingchentsai...,https://www.104.com.tw/job/?jobno=54ynp&jobsou...
7,聯強國際股份有限公司,聯強 資料分析師(亞太第一大資通訊通路集團),(2019菁英計劃)於集團總部負責全球各地營運系統之研發與管理。每年投資7億打造國際級軟體之...,"['統計學研究員', '數學研究員', '電腦系統分析師']","月薪30,000~70,000元",全職,台北市中山區民生東路三段75號,不需負擔管理責任,無需出差外派,日班，09:00~18:00,...,大學以上,不拘,不拘,不拘,不拘,1. 具溝通協調與分析能力\r2. 能獨立處理及解決問題\r3. 具程式開發經驗者佳\r4....,【聯強的人才培育理念】\r 更多聯強營運及人才發展過程請關注下方連結：\rhttps://y...,統計學相關、其他數學及電算機科學相關,聯絡人：人資行政部其他：應徵流程說明 \r1.筆試(通過初審者通知筆試) \r2.面試(通過...,https://www.104.com.tw/job/?jobno=6hy11&jobsou...
8,正積材料有限公司,業務工程師 Sales Engineer,1. 客戶經營：維護現有客戶，開發新客戶，客製化優質服務與解決方案\r2. 業務拓展：產品推...,"['國內業務人員', '國內業務主管', '業務支援工程師']",待遇面議（經常性薪資達4萬元或以上）,全職,台北市信義區松高路9號27樓,不需負擔管理責任,需出差，一年累積時間約一個月以下,日班,...,大學以上,不拘,\n英文 -- 聽 /中等、說 /中等、讀 /中等、寫 /中等\n,Adobe Acrobat、Excel、Internet Explorer、PowerPoi...,不拘,普通小型車,一、薪資福利最實在：\r ★週休二日\r ★年假/特休\r ...,工程學科類、自然科學學科類、國際貿易相關,聯絡人：Julia WeiE-mail：hr@jmat.co其他：面試時請準備簡短自我介紹，...,https://www.104.com.tw/job/?jobno=66gx9&jobsou...
9,正積材料有限公司,業務助理 Sales Associate,1.\t行政庶務：Email回覆、電話接聽、信件收發、文件整理歸檔\r2.\t業務協助：訂單...,"['國內業務人員', '業務助理', '國外業務人員']",待遇面議（經常性薪資達4萬元或以上）,全職,台北市信義區松高路9號27樓,不需負擔管理責任,需出差，一年累積時間約三個月以下,日班，09:00-18:00,...,大學、碩士,不拘,\n英文 -- 聽 /中等、說 /中等、讀 /中等、寫 /中等\n,

In [11]:
JobList.to_excel('C:/Users/TL_Yu/Desktop/JobList2.xlsx', encoding='cp950')